**Loading and Preprocessing - setting up for cross validation**

In [49]:
import time
import numpy as np
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import KFold

# centre the data
def centre_data(train, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    return train, test

# apply PCA on the data 
def PCA(variance_target, training_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components])



def load_in_dataset_and_preprocess(explained_variance, training_data, testing_data, training_labels,testing_labels):
    

    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')

    # Centre data
    #training_data, testing_data= centre_data(training_data, testing_data)

    # Apply PCA
    #training_data, testing_data = PCA(explained_variance, training_data, testing_data)

    number_training_samples = len(training_data)
    number_testing_samples = len(testing_data)

    # Reshape data from channel to rows
    training_data = np.reshape(training_data, (number_training_samples, -1))
    testing_data = np.reshape(testing_data, (number_testing_samples, -1))

    return training_data, training_labels, testing_data, testing_labels

**Support Vector Machine**

In [50]:
import time
import numpy as np
import numpy as np
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import KFold

class SVM:

    training_data=[]
    testing_data=[]

    def __init__(self, training_data, testing_data):
      self.training_data= training_data
      self.testing_data= testing_data

    def __len__(self, data):
      return len(data)

    def calculate_linear_output(self, data, weights):
      return np.dot(data, weights)

    # distance of point from separating hyperplane?
    def calculate_distance(self, X, w):
      return  y * (np.dot(X, w)) - 1

    # calculate gradient, use L2 regularisation 
    def calculate_gradient(self, weights, regularisation_param, training_data, training_labels, num_classes):
        
        num_training_samples, num_training_features = training_data.shape
        
        gradient = np.zeros((num_training_features, num_classes))
        
        # w^Tx
        linear_output = self.calculate_linear_output(training_data, weights)

        #linear output with labels 
        linear_output_y_i = linear_output[np.arange(num_training_samples),training_labels]
        delta = linear_output - linear_output_y_i[:,np.newaxis] + 1
        
        ones_and_zeros = np.zeros(delta.shape)
        
        # makes all the places where delta > 0, 1 else 0
        # With lagrange multiplier considered, if the sample is on the support vector: 𝛼 = 1
        # else: 𝛼 = 0
        ones_and_zeros = np.where(delta > 0, 1, 0)
        
        # calculate the sum of each row 
        sum_of_each_row = np.sum(ones_and_zeros, axis=1)
        
        ones_and_zeros[np.arange(num_training_samples), training_labels] = - sum_of_each_row

        gradient = (1/num_training_samples) * np.dot((training_data.T), ones_and_zeros)
        
        # controls the influence of each individual support vector on the objective function. 
        # Greater C decreases the effect of |w|²/2, and results in the narrower margin
        gradient = gradient + (2* regularisation_param * weights)
        
        return gradient 

    # train model using stochastic gradient descent 
    def train_model(self, training_data, training_labels, weights, learning_rate, regularisation_param, iterations, batch_size, num_classes):
      
      num_training_samples = len(training_data)
      weights = weights

      for i in range(iterations):
      # create batch
          batch = np.random.choice(5000, batch_size) #change this to num_training_samples 
          gradient = self.calculate_gradient(weights, regularisation_param, training_data[batch], training_labels[batch], num_classes)
          weights = weights - learning_rate * gradient

      return weights

    # calculate accuracy of model 
    def calculate_accuracy (self, data, labels, weights):
        
        accuracy = 0
        prediction = np.zeros(len(data))

      #w^Tx
        linear_output= self.calculate_linear_output(data, weights)

      # returns the indices of the maximum values along an axis, ie. in this case will return the 
      # column index corresponding to the greatest index of each row
        prediction = np.argmax(linear_output, axis=1)

      # count the number of predictions that are correct 
        total_correct_predictions = (prediction == labels).sum()
        num_data_points = len(data)
        accuracy = (total_correct_predictions/num_data_points)*100

        return accuracy


#using 10 fold cross validation here to evaluate the performance of SVM
def cross_validation():

  (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
  cv = KFold(n_splits=10)
  i=0
  j=0

  for train, test in cv.split(training_data):
    
    for train_label, test_label in cv.split(training_data):

      if(i!=j):
        j+=1
        continue

      else:
        training_data, training_labels, testing_data, testing_labels = load_in_dataset_and_preprocess(0.9, train, train_label, test, test_label)
        svm = SVM(training_data, testing_data)
        num_classes = np.max(training_labels) + 1
        weights = np.ones((len(training_data[1]), num_classes))
        weights= svm.train_model(training_data, training_labels, weights, 0.00000001, 1000, 20000, 200, 20)

        total_accuracy = svm.calculate_accuracy(testing_data, testing_labels, weights)
        print('accuracy: ', total_accuracy)
        break
    i+=1

cross_validation()

  #run the classifiers here 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


AttributeError: ignored

 for train, test in zip(cv.split(training_data), cv.split(training_labels)):
   
    training_data, training_labels, testing_data, testing_labels = load_in_dataset_and_preprocess(0.9, train[0], test[0], train[1], test[1])

    svm = SVM(training_data, testing_data)
    num_classes = np.max(training_labels) + 1
    weights = np.ones((len(training_data[1]), num_classes))
    weights= svm.train_model(training_data, training_labels, weights, 0.00000001, 1000, 20000, 200, 20)

    total_accuracy = svm.calculate_accuracy(testing_data, testing_labels, weights)
    print('accuracy: ', total_accuracy)
    break

**Multi-Layer Perceptron**

In [52]:
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical  
from keras.datasets import cifar100
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout
import numpy as np

def run_MLP_model(training_data, training_labels, testing_data, testing_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay_level, momentum, epochs, loss_function):

    model = Sequential()

    model.add(Dense(num_hidden_units, activation=first_activation_function, input_dim=training_data.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden_units, activation=first_activation_function))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation=second_activation_function))

    if (optimiser == 'SGD'):
        op = SGD(lr=learning_rate, decay=decay_level, momentum=momentum, nesterov=True)

    else:
        op = Adam(lr=learning_rate, decay=decay_level)

    # can also use loss function categorical_crossentropy
    # or optimiser SGD
    # try with different optimisers and loss functions
    model.compile(optimizer=op,
                  loss=loss_function,
                  metrics=['accuracy'])

    history = model.fit(training_data, training_labels, epochs=epochs, batch_size=32, verbose=0, validation_split=0.2)

    score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)
    return score[1]

# helper function for concatenating labels onto their corresponding data points
def concatenate_data(training_data, training_labels):
    return np.column_stack((training_data, training_labels))

def cross_validation():

  (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
  cv = KFold(n_splits=10)
  momentum = 0.9
  decay=1e-6
  learning_rate = 0.001
  first_activation_function = 'relu'
  second_activation_function = 'softmax'
  loss = 'sparse_categorical_crossentropy'
  optimiser = 'Adam'
  epochs = 20
  num_hidden_units = 256

  i=0
  j=0

  for train, test in cv.split(training_data):
    
    for train_label, test_label in cv.split(training_data):

      if(i!=j):
        j+=1
        continue

      else:
        training_data, training_labels, testing_data, testing_labels = load_in_dataset_and_preprocess(0.9, train, train_label, test, test_label)
        run_MLP_model(training_data, training_labels, testing_data, testing_labels, first_activation_function, second_activation_function, num_hidden_units, learning_rate, optimiser, decay, momentum, epochs, loss)

        break
    i+=1

cross_validation()

ValueError: ignored

**Random Forests**

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt

def import_data() -> ((np.ndarray, np.ndarray), (np.ndarray, np.ndarray)):
    return tf.keras.datasets.cifar100.load_data(label_mode="coarse")
# (x_train, y_train), (x_test, y_test) = import_data()

# x_train, x_test = convert_to_grayscale(x_train, x_test)

# x_train, x_test = increase_all_contrast(x_train, x_test, 10)

# x_train, x_test = equalize_hist_all(x_train, x_test)

# x_train, x_test = equalize_adapthist_all(x_train, x_test, 0.03)

# x_train, x_test = canny_edge_filter_all(x_train, x_test, 0)


# x_train, x_test = centre_data(x_train, x_test)

# x_train, x_test = pca(x_train, x_test, 0.8)


def cross_validation():

  (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
  cv = KFold(n_splits=10)
  momentum = 0.9
  decay=1e-6
  learning_rate = 0.001
  first_activation_function = 'relu'
  second_activation_function = 'softmax'
  loss = 'sparse_categorical_crossentropy'
  optimiser = 'Adam'
  epochs = 20
  num_hidden_units = 256

  for train, test in zip(cv.split(training_data), cv.split(training_labels)):
   
    training_data, training_labels, testing_data, testing_labels = load_in_dataset_and_preprocess(0.8, train[0], train[1], test[0], test[1])
    x_train, y_train, x_test, y_test = flatten_data(train[0], test[0], train[1], test[1])

    model = RandomForestClassifier(
    n_jobs=-1, 
    verbose=1,
    n_estimators=100,
    bootstrap=False, 
    max_features='log2', 
    criterion='gini')

    model.fit(x_train, y_train)

    model.score(x_test, y_test)

cross_validation()